In [2]:
# importing libraries
import pandas as pd
import numpy as np  
import re

############### Question 1 ###############

In [2]:
def check_sum(li, num):
    for i in range(len(li)-1):
        for j in range(i+1,len(li)):
            sum_of_2 = li[i]+li[j]
            if num == sum_of_2:
                return True
    return False

# Order of (n*(n-1))/2

In [3]:
## USER INPUT
li = [13, 6, 8, 22, 3]
# li = [13, 6, 8]
num = 14
check_sum(li, num)

True

############### Question 5 ###############

In [4]:
def match_percent_exact(w, t):
    text_len = len(t.split())
    word_count = 0
    for i in t.split():
        if w == i:
            word_count += 1
    perc = round(100*word_count/text_len,2)
    return ("% of word to word match: {}".format(perc))
    
def match_percent_fuzzy(w, t):
    text_len = len(t.split())
    word_count = 0
    for i in t.split():
        if w in i:
            word_count += 1
    perc = round(100*word_count/text_len,2)
    return ("% of word to text match: {}".format(perc))

In [5]:
## INPUT HERE
word = "rent"
text = "Sam rented his car so he could rent a boat for vacation"

print(match_percent_exact(word, text))
print(match_percent_fuzzy(word, text))

% of word to word match: 8.33
% of word to text match: 16.67


In [6]:
# Assumptions for Question 5:
# 'how often a given word occurs in a text' may mean:
    # 1. the word may exist in text as a complete word
        # match_percent_exact() is for this case
    # 2. the word may exist in text as a part of another word (or complete word)
        # match_percent_fuzzy() is for this case

############### Problem (Solution 2) ###############

In [1]:
# Helper Functions
import pandas as pd
import numpy as np  
import re
import itertools
import gc
import itertools as it
import more_itertools as mit
import time

def get_2(strx):
    strx1 = strx[0:2]
    strx = strx[2:len(strx)]
    return (strx1, strx)

def get_3(strx):
    strx1 = strx[0:3]
    strx = strx[3:len(strx)]
    return (strx1, strx)

def get_numbers(s):
    numbers = []
    for i in range(int(len(s)/2)):
        numbers.append(2)
    for i in range(int(len(s)/3)):
        numbers.append(3)
    return numbers

def detect_seq(lx1):
    for i in range(0, len(lx1)-1):
        if lx1[i] == lx1[i+1]:
            return True
    return False

def get_final_substr(comb, l_r, center, side):
    # converting all combinations of 2 and 3 to sub_strings
    list_str2 = []
    gc.collect()
    for i in comb:
        if i == 2:
            strx1, l_r = get_2(l_r)
            list_str2.append(strx1)
        else:
            strx1, l_r = get_3(l_r)
            list_str2.append(strx1)
    if side == 'l':
        list_xx = list_str2 + [center]
    else:
        list_xx = [center] + list_str2
    
    # removing sub_string sequence with consecutive sub_strings like 'cc' and 'cc'
    if detect_seq(list_xx) == False:
        return list_str2
    else:
        return []
        
def Combinations(numbers, target, partial=[], partial_sum=0):
    """get all combinations of 2 & 3 which sum to len(input_str) - one by one"""
    if partial_sum == target:
        yield partial
    if partial_sum >= target:
        return
    for i, n in enumerate(numbers):
        remaining = numbers[i + 1:]
        yield from Combinations(remaining, target, partial + [n], partial_sum + n)

def Permutations(comb):
    """get permutations of 2 & 3 for each 'combination' from combinations - one by one"""
    Permutations = []
    for i in mit.distinct_permutations(comb):
        yield i
        
# generate permutations of a list numbers adding to a constant
def get_combo(side, l_r, center): # l_r is left/right string; side='l'/'r'
    # get all combinations of 2/3 which sum to len(input_str)
    # implementing with generator
    Perm_final = []
    numbers = get_numbers(l_r)
    c = Combinations(numbers, len(l_r))
    result_1 = 1
    while result_1 is not None:
        try:
            result_1 = next(c)
        except:
            break
    
        comb = result_1
            
        xx = Permutations(comb)
        y=1
        while y is not None:
            try:
                y = next(xx)
                sub_str = get_final_substr(y, l_r, center, side)
                if sub_str != []:
                    Perm_final = sub_str
                    Perm_final = list(set(Perm_final))
                    return Perm_final
            except:
                break
    return Perm_final

def get_left_center2_right(s,i):
    s_center = s[i:i+2]
    try:
        s_left = s[0:i]
    except:
        s_left = ''
    try:
        s_right = s[i+2:len(s)]
    except:
        s_right = ''
    return (s_left, s_center, s_right)

def get_left_center3_right(s,i):
    s_center = s[i:i+3]
    try:
        s_left = s[0:i]
    except:
        s_left = ''
    try:
        s_right = s[i+3:len(s)]
    except:
        s_right = ''
    return (s_left, s_center, s_right)

In [2]:
def split_string(strx):

    ## Find and add all substrs left/right of every 2 length substr
    Final_List_2 = []
    for i in range(len(strx)-1):
        left, center, right = get_left_center2_right(strx, i)

        combo_left = get_combo('l', left, center)
        combo_right = get_combo('r', right, center)


        if center not in Final_List_2:

            if i == 0:
                if combo_right != []:
                    Final_List_2 += combo_right
                    Final_List_2.append(center)

            elif i == len(strx)-2:
                if combo_left != []:
                    Final_List_2 += combo_left
                    Final_List_2.append(center)

            else:
                if combo_left!=[] and combo_right!=[]:
                    Final_List_2 += combo_left
                    Final_List_2 += combo_right
                    Final_List_2.append(center)

    Final_List_2 = list(set(Final_List_2))
    
    
    ## Find and add all substrs left/right of every 3 length substr
    Final_List_3 = []
    for i in range(len(strx)-2):
        left, center, right = get_left_center3_right(strx, i)

        combo_left = get_combo('l', left, center)
        combo_right = get_combo('r', right, center)

        if center not in Final_List_3:

            if i == 0:
                if get_combo('r', right, center) != []:
                    Final_List_3 += combo_right
                    Final_List_3.append(center)

            elif i == len(strx)-3:
                if get_combo('l', left, center) != []:
                    Final_List_3 += combo_left
                    Final_List_3.append(center)

            else:
                if get_combo('l', left, center) != [] and get_combo('r', right, center) != []:
                    Final_List_3 += combo_left
                    Final_List_3 += combo_right
                    Final_List_3.append(center)

    Final_List_3 = list(set(Final_List_3))
    
    Final_List = Final_List_2 + Final_List_3
    Final_List = list(set(Final_List))
    return Final_List

In [3]:
# strx = "abcdefg"
# strx = "ccccacccc"
# strx = 'ababcabc'
# strx = 'abcdefghijklmnopqrstuvwxyabcde' # 30
strx = 'abcdefghijklmnopqrstuvwxyabcdegtfrghbhgnnlposhgtdk' # 50
# strx = 'abcdefghijklmnopqrstuvwxyabcdegtfrghbhgnnlposhgtdkabcdefghijklmnopqrstuvwxyabcdegtfrghbhgnnlposhgtdk' # 100


start_time = time.time()

# split_string function
print("String Length: {}".format(len(strx)))
output = split_string(strx)
print("Output: {}".format(output))

elapsed_time = time.time() - start_time
print("Seconds: {}".format(elapsed_time))

String Length: 50
Output: ['hb', 'opq', 'tfr', 'wx', 'de', 'hij', 'ghi', 'qr', 'nnl', 'uvw', 'mn', 'lpo', 'os', 'lm', 'vw', 'pos', 'rst', 'rg', 'ef', 'deg', 'gh', 'ij', 'nlp', 'tf', 'dk', 'gn', 'pq', 'osh', 'bhg', 'sh', 'egt', 'efg', 'gnn', 'abc', 'cde', 'stu', 'uv', 'gt', 'hg', 'nop', 'mno', 'tu', 'fgh', 'hbh', 'nl', 'vwx', 'no', 'lmn', 'yab', 'ghb', 'cd', 'fr', 'klm', 'nn', 'xy', 'wxy', 'bc', 'rs', 'fg', 'rgh', 'pqr', 'hgn', 'st', 'hgt', 'ijk', 'gtf', 'jkl', 'hi', 'tdk', 'tuv', 'op', 'bh', 'bcd', 'shg', 'ya', 'lp', 'eg', 'jk', 'qrs', 'frg', 'kl', 'def', 'xya', 'po', 'ab']
Seconds: 6.783310174942017
